Matrix element (weakfield) - BW version
========
Testenv to check the valitity of the matrix element calculation on a grid.

In [1]:
from __future__ import print_function
import sftrident as sf
import sftrident.coordinate as corr
from sftrident.matrixElement2 import matrixElementWF
import numpy as np
import time
times = {}

In [2]:
def buildArr_lcc(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]


def buildArr_sph(Nss,Ne1,Nc1,Np1,Ne2,Nc2,Np2,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    e1 = np.linspace(1.0+eps,5,Ne1)
    c1 = np.linspace(-1.0,1.0,Nc1)
    p1 = np.linspace(0.0,2.0*np.pi,Np1)
    e2 = np.linspace(1.0+eps,5,Ne2)
    c2 = np.linspace(-1.0,1.0,Nc2)
    p2 = np.linspace(0.0,2.0*np.pi,Np2)
    return [ss,e1,c1,p1,e2,c2,p2]


mass = 1.0
Narr1 = (2,3,4,5,6,7,8)
frame = 'lab'
tempDPHI = 50.0

arrLCC = buildArr_lcc(*Narr1)

arrSPH = buildArr_sph(*Narr1)

start = time.time()
#kinObj = corr.momenta(*arrLCC,mass = mass,frame = frame,coordSys='lcc')
kinObj = corr.momenta(*arrSPH,mass = mass,frame = frame,coordSys='sph')
end =  time.time() - start
times['initMom']=end

config = {'a0':1e-4,'mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}

start = time.time()
matObj = matrixElementWF(kinObj,**config)
end =  time.time() - start
times['initMat'] = end

print(matObj.bw(0,1,1,0)[0])



(6.378392537985453e-11-0j)


In [3]:
spins = [(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)]

wrappBW = lambda spn : np.asarray(matObj.bw(*spn))

start = time.time()
resBWgrid = np.array(map(wrappBW,spins))
end =  time.time() - start
times['mapMat'] =end

print(resBWgrid.shape)

(16, 960)


In [4]:
def sorted2List(X,Y):
    """
    sorts the list X with the values of Y
    """
    return [x for _, x in sorted(zip(Y,X), key=lambda pair: pair[0])]


modeDict={
'bw':[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15],
'c':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
'bwx':[0, 11, 8, 3, 14, 5, 6, 13, 2, 9, 10, 1, 12, 7, 4, 15],
'cx':[0, 14, 2, 12, 11, 5, 9, 7, 8, 6, 10, 4, 3, 13, 1, 15]
}
spinMode = np.array([
    (0,0,0,0),
    (0,0,1,1),
    (0,0,0,1),
    (0,0,1,0),
    (1,1,0,0),
    (1,1,1,1),
    (1,1,0,1),
    (1,1,1,0),
    (0,1,0,0),
    (0,1,1,1),
    (0,1,0,1),
    (0,1,1,0),
    (1,0,0,0),
    (1,0,1,1),
    (1,0,0,1),
    (1,0,1,0),
])
    
matObjSQ = sf.matrixElement(config)
def getMatBW(n):
    kinPara = [kinObj.grid.ssGrid[n],kinObj.grid.lccGrid1[1][n], kinObj.grid.lccGrid1[2][n],kinObj.grid.lccGrid1[0][n],kinObj.grid.lccGrid2[1][n], kinObj.grid.lccGrid2[2][n],kinObj.grid.lccGrid2[0][n]]
    matObjSQ.setKin(kinPara)
    resOld = matObjSQ.evalMdirPartBW()
    tempSpins = spinMode[modeDict['bw']]
    #print(sorted2List([list(t) for t in tempSpins],[list(t) for t in tempSpins]))
    return sorted2List(np.asarray(resOld),[list(t) for t in tempSpins])
    #return kinPara

err = 0
maxAERR_R = 0.0
maxAERR_I = 0.0

relERR_R = 0.0
maxRERR_R = 0.0
relERR_I = 0.0
maxRERR_I = 0.0

Msq_aerr=0.0
Msq_rerr = 0.0
Msq_rerr_max = 0.0

for ind in np.arange(kinObj.grid.size):
    bwResSQ = getMatBW(ind)
    MsqSQ = 0.0
    MsqNP = 0.0
    for i,spin in enumerate(spins):
        #print('i',i)
        #print()
        #print(el)
        #print("\t",bwResSQ[i])
        #print("\t",resBWgrid[i][ind])
        test = np.isclose(bwResSQ[i],resBWgrid[i][ind],atol=1e-15,rtol=1e-8)
        absERR_R = bwResSQ[i].real-resBWgrid[i][ind].real
        maxAERR_R = max(maxAERR_R,absERR_R)
        norm_R = bwResSQ[i].real+resBWgrid[i][ind].real
        if norm_R != 0.0:
            relERR_R = absERR_R/norm_R
        maxRERR_R = max(maxRERR_R,relERR_R)
        
        absERR_I = bwResSQ[i].imag-resBWgrid[i][ind].imag
        maxAERR_I = max(maxAERR_I,absERR_I)
        norm_I = bwResSQ[i].imag+resBWgrid[i][ind].imag
        if norm_I != 0.0:
            relERR_I = absERR_I/norm_I
        maxRERR_I = max(maxRERR_I,relERR_I)
        #if not test:
            #print("ind",ind)
            #print('i',i)
            #print(spin)
            #print(spins)
            #print("\t",bwResSQ[i])
            #print("\t",resBWgrid[i][ind])
            #err+=1
        MsqSQ += np.real(bwResSQ[i]*np.conjugate(bwResSQ[i]))
        MsqNP += np.real(resBWgrid[i][ind]*np.conjugate(resBWgrid[i][ind]))
    Msq_aerr = max(np.abs(MsqSQ-MsqNP),Msq_aerr)
    Msq_norm =MsqSQ+MsqNP
    if Msq_norm!=0.0:
        Msq_rerr = np.abs(MsqSQ-MsqNP)/Msq_norm
        
    Msq_rerr_max = max(Msq_rerr,Msq_rerr_max)
    if Msq_rerr>1.0:
        print("relErr to high:")
        print(ind)
        print("\t",Msq_rerr)
        print("\t",MsqSQ)
        print("\t",MsqNP)
        
    
    if not np.isclose(MsqSQ,MsqNP):
        print(ind)
        print("\t",MsqSQ)
        print("\t",MsqNP)
        err+=1
        
print("Done with %d errors."%err)
print("max. abs. error: %1.2e + i %1.2e"%(maxAERR_R,maxAERR_I))
print("max. rel. error: %1.2e + i %1.2e"%(maxRERR_R,maxRERR_I))
print("max. abs. error (Msq): %1.2e"%(Msq_aerr))
print("max. rel. error (Msq): %1.2e"%(Msq_rerr_max))

Done with 0 errors.
max. abs. error: 1.39e-17 + i 5.62e-23
max. rel. error: 8.78e-15 + i 5.74e-15
max. abs. error (Msq): 6.94e-18
max. rel. error (Msq): 2.55e-16


In [5]:
for el in times.iterkeys():
    print(el,times[el]/(kinObj.grid.size*1.0))

mapMat 0.000481375058492
initMom 3.11603148778e-05
initMat 7.20761716366e-05


Conclusion
===

There are some issues with the spin mapping in sftrident.matrixElement. The results are (mostly) exactly the same, but with a overall minus. Therefore, the squared matrixElement is not influenced from this bug. But there need to be some research, where the bug comes from and how one may solve it. 

Overall, one may conclude, that the BW matrix element is computed with a grid evaluation and gives almost the same results as the serial evaluation (modulo sign). 